In [2]:
import json
import boto3
import base64
s3 = boto3.client('s3')

def lambda_handler(event, context):
    key =event["s3_key"]
    bucket = event["s3_bucket"]
    print(key)

    boto3.resource('s3').Bucket(bucket).download_file(key, "/tmp/image.png")

    # We read the data from a file
    with open("/tmp/image.png", "rb") as f:
        image_data = base64.b64encode(f.read())

    # Pass the data back to the Step Function
    print("Event:", event.keys())
    return {
        'statusCode': 200,
        'body': {
            "image_data": image_data,
            "s3_bucket": bucket,
            "s3_key": key,
            "inferences": []
        }
}

In [3]:
import json
import base64
import boto3
ENDPOINT = "new-endpoint"
def lambda_handler(event, context):
    image = base64.b64decode(event["body"]["image_data"])
    runtime = boto3.client('sagemaker-runtime')
    response = runtime.invoke_endpoint(
        EndpointName=ENDPOINT,
        ContentType='application/x-image',  
        Body=image
    )

    inferences = json.loads(response['Body'].read().decode('utf-8'))
    event["inferences"] = inferences
    return {
        'statusCode': 200,
        'body': {
            "image_data": event["body"]['image_data'],
            "s3_bucket": event["body"]['s3_bucket'],
            "s3_key": event["body"]['s3_key'],
            "inferences": event['inferences'],
       }
    }


In [4]:
import json

THRESHOLD = 0.9

def lambda_handler(event, context):
    inferences = event["body"]['inferences']
    meets_threshold = max(list(inferences))>THRESHOLD

    if meets_threshold:
        pass
    else:
        raise("THRESHOLD_NOT_MET")

    return {
        'statusCode': 200,
        'body': event
    }